<a href="https://colab.research.google.com/github/Jsebastianmm1/Bootcamp_programaci-n/blob/main/PROYECTO_COMPUTACI%C3%93N_DEFINITIVO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **¿Cómo usar este script?**
1. Para que el código corra correctamente, es necesario ejecutarlo en google colab, dado que para las entradas de archivos kmz se usa una libreria especial de colab.
2. Al estar en google colab, en la parte superior, dar click en "Run all", esto con el fin de que corra todos los bloques de código en el orden adecuado, y se puedan activar todas las funcionalidades.


In [ ]:
!pip install pykml lxml shapely requests pyproj timezonefinder pytz

In [ ]:
import zipfile
import requests
from lxml import etree
from pykml import parser
from pykml.factory import KML_ElementMaker as KML
from google.colab import files
from shapely.geometry import Polygon, LineString
from pyproj import Geod
from timezonefinder import TimezoneFinder

#ANALISIS DE COORDENADAS
def obtener_elevacion(lat, lon):
    try:
        url = f"https://api.open-meteo.com/v1/elevation?latitude={lat}&longitude={lon}"
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        return response.json()['elevation'][0]
    except:
        return None

def analizar_punto_basico(punto, nombre="Punto"):
    lon, lat = punto
    print(f"\n--- 🔎 Análisis Básico del Punto ---")
    print(f"  - {nombre}:")
    print(f"    -> Coordenadas (Lon, Lat): ({lon:.6f}, {lat:.6f})")
    elevacion = obtener_elevacion(lat, lon)
    if elevacion is not None:
        print(f"    -> Altura sobre el nivel del mar: {elevacion} metros")
    print("\n    ➤ Copia las coordenadas de arriba para usarlas en los siguientes bloques de análisis.")

def analizar_poligonos_basico(lista_de_poligonos):
    geod = Geod(ellps='WGS84')
    for i, vertices in enumerate(lista_de_poligonos):
        poly_shape = Polygon(vertices)
        area, _ = geod.geometry_area_perimeter(poly_shape)
        centroide = poly_shape.centroid
        print(f"\n--- 🔎 Análisis del Polígono {i+1} ---")
        print(f"    -> Área: {abs(area):,.2f} metros cuadrados")
        analizar_punto_basico(centroide.coords[0], nombre=f"Centroide del Polígono {i+1}")

def analizar_rutas_basico(lista_de_rutas):
    for i, vertices in enumerate(lista_de_rutas):
        linea_shape = LineString(vertices)
        punto_medio = linea_shape.interpolate(0.5, normalized=True)
        print(f"\n--- 🔎 Análisis de la Ruta {i+1} ---")
        analizar_punto_basico(punto_medio.coords[0], nombre=f"Punto Medio de la Ruta {i+1}")

# LEE KMZ
def leer_kmz():
    print("\n--- 📂 Leer Archivo KMZ ---")
    uploaded = files.upload()
    if not uploaded: return
    nombre_archivo = list(uploaded.keys())[0]
    print(f"\nProcesando '{nombre_archivo}'...")
    datos = obtener_coordenadas_kmz(nombre_archivo)
    if datos["puntos"]:
        for i, punto in enumerate(datos["puntos"]):
            analizar_punto_basico(punto, nombre=f"Punto {i+1} del archivo")
    if datos["rutas"]:
        analizar_rutas_basico(datos["rutas"])
    if datos["poligonos"]:
        analizar_poligonos_basico(datos["poligonos"])
    if not any(datos.values()):
        print("\nNo se encontraron elementos analizables en el archivo.")

def obtener_coordenadas_kmz(archivo_kmz):
    coordenadas = {"puntos": [], "rutas": [], "poligonos": []}
    try:
        with zipfile.ZipFile(archivo_kmz, 'r') as kmz:
            kml_file = next((s for s in kmz.namelist() if s.lower().endswith('.kml')), None)
            if not kml_file: return coordenadas
            kml_obj = parser.fromstring(kmz.read(kml_file))
            ns = '{http://www.opengis.net/kml/2.2}'
            for p in kml_obj.findall('.//' + ns + 'Point'):
                parts = p.coordinates.text.strip().split(',')
                coordenadas['puntos'].append((float(parts[0]), float(parts[1])))
            for p in kml_obj.findall('.//' + ns + 'LineString'):
                coordenadas['rutas'].append([tuple(map(float, c.split(',')[:2])) for c in p.coordinates.text.strip().split()])
            for p in kml_obj.findall('.//' + ns + 'Polygon'):
                coordenadas['poligonos'].append([tuple(map(float, c.split(',')[:2])) for c in p.outerBoundaryIs.LinearRing.coordinates.text.strip().split()])
    except Exception as e:
        print(f"\nOcurrió un error al leer el archivo: {e}")
    return coordenadas

#CREA KMZ
def menu_creacion():
    while True:
        print("\n--- 📝 Menú de Creación ---")
        print("1. Crear un Punto 📍\n2. Crear una Ruta 🛤️\n3. Crear un Polígono 🗺️\n4. Volver")
        opcion = input("\nElige una opción: ")
        if opcion in ['1','2','3']: crear_elemento({'1':'Punto','2':'Ruta','3':'Poligono'}[opcion])
        elif opcion == '4': break
        else: print("\nOpción no válida.")

def crear_elemento(tipo):
    print(f"\n--- Creando: Nuevo {tipo} ---")
    nombre = input(f"➤ Ingresa el nombre del {tipo}: ")
    coords_input = input("➤ Ingresa las coordenadas (longitud,latitud): ")
    coordenadas = parsear_coordenadas(coords_input)
    if validar_coordenadas(coordenadas, tipo):
        guardar_kmz(nombre, coordenadas, tipo)
        if tipo == 'Punto': analizar_punto_basico(coordenadas[0])
        elif tipo == 'Ruta': analizar_rutas_basico([coordenadas])
        elif tipo == 'Poligono': analizar_poligonos_basico([coordenadas])

def parsear_coordenadas(input_str):
    try: return [tuple(map(float, p.split(','))) for p in input_str.strip().replace('\n', ' ').split(' ') if p]
    except (ValueError, IndexError): print("\n❌ ERROR: Formato inválido."); return None

def validar_coordenadas(coordenadas, tipo):
    if not coordenadas: return False
    if tipo == 'Punto' and len(coordenadas) != 1: print("\n❌ ERROR: Un punto debe tener un solo par de coordenadas."); return False
    if tipo == 'Ruta' and len(coordenadas) < 2: print("\n❌ ERROR: Una ruta necesita al menos dos puntos."); return False
    if tipo == 'Poligono' and (len(coordenadas) < 4 or coordenadas[0] != coordenadas[-1]): print("\n❌ ERROR: Un polígono necesita al menos 4 puntos y el primero debe ser igual al último."); return False
    return True

def guardar_kmz(nombre, coordenadas, tipo):
    nombre_archivo = input("➤ Ingresa el nombre para el archivo de salida (ej: mi_mapa.kmz): ")
    if not nombre_archivo.lower().endswith('.kmz'): nombre_archivo += '.kmz'
    placemark = KML.Placemark(KML.name(nombre))
    coords_str = " ".join([f"{lon},{lat},0" for lon, lat in coordenadas])
    if tipo == 'Punto': geom = KML.Point(KML.coordinates(coords_str))
    elif tipo == 'Ruta': geom = KML.LineString(KML.tessellate(1), KML.coordinates(coords_str))
    else: geom = KML.Polygon(KML.outerBoundaryIs(KML.LinearRing(KML.coordinates(coords_str))))
    placemark.append(geom)
    doc = KML.kml(KML.Document(placemark))
    with zipfile.ZipFile(nombre_archivo, 'w') as kmz: kmz.writestr('doc.kml', etree.tostring(doc, pretty_print=True))
    print(f"\n✅ ¡Éxito! Se ha guardado el archivo '{nombre_archivo}'.")

# --- Menú Principal del Bloque 1 ---
def menu_principal():
    while True:
        print("\n=============================================")
        print("      INGRESA TUS COORDENADAS     ")
        print("=============================================")
        print("1. 📂 Leer y Analizar un archivo KMZ")
        print("2. 📝 Crear un archivo KMZ")
        print("3. 🚪 Salir")
        opcion = input("\n¿Qué te gustaría hacer? Elige una opción: ")
        if opcion == '1': leer_kmz()
        elif opcion == '2': menu_creacion()
        elif opcion == '3': print("\n¡Programa finalizado!"); break
        else: print("\nOpción no válida.")

menu_principal()

In [ ]:
!pip install skyfield matplotlib numpy pytz timezonefinder

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from timezonefinder import TimezoneFinder
import pytz
from skyfield.api import Topos, load

def calcular_posiciones_solares(lat, lon, year, month, day):
    """
    Calcula la posición del sol (altitud y azimut) para un día y lugar específicos.
    Usa una resolución de 5 minutos para generar curvas suaves.
    """
    print(f"\nCalculando posiciones del sol para el {day}/{month}/{year}...")
    ts, eph = load.timescale(), load('de421.bsp')
    observer = eph['earth'] + Topos(latitude_degrees=lat, longitude_degrees=lon)

    #POSICIONES
    posiciones = []
    for h in range(24):
        for m in range(0, 60, 5):
            t = ts.utc(year, month, day, h, m)
            alt, az, _ = observer.at(t).observe(eph['sun']).apparent().altaz()
            if alt.degrees > 0:
                posiciones.append({'time': t, 'azimuth': az.degrees, 'altitude': alt.degrees})

    if not posiciones:
        print("El sol no está sobre el horizonte en la fecha y ubicación especificadas.")
    return posiciones

def graficar_analisis_solar(posiciones, fecha_str, lat, tz_name):
    """
    Crea una única figura con dos subgráficos: la trayectoria polar y la curva de altitud.
    """
    print("Generando gráficas de análisis solar...")

    #EXTRACCIÓN DATOS
    times_utc = [p['time'] for p in posiciones]
    azimuts = np.array([p['azimuth'] for p in posiciones])
    altitudes = np.array([p['altitude'] for p in posiciones])


    fig = plt.figure(figsize=(18, 8))
    fig.suptitle(f"Análisis Solar para {fecha_str} | Latitud: {lat:.2f}°", fontsize=16)

    #TRAYECTORIA POLAR
    ax_polar = fig.add_subplot(1, 2, 1, projection='polar')

    az_rad = np.deg2rad(azimuts)
    radio = 90 - altitudes

    ax_polar.plot(az_rad, radio, color='blue', linewidth=2)
    ax_polar.set_title("Trayectoria Solar (Vista Cenital)", pad=20)

    # EJES
    ax_polar.set_theta_offset(np.pi / 2.0)
    ax_polar.set_theta_direction(-1)
    ax_polar.set_xticks(np.deg2rad([0, 90, 180, 270]))
    ax_polar.set_xticklabels(['E', 'N', 'O', 'S'])
    ax_polar.set_rlim(0, 90)
    ax_polar.set_rgrids(range(0, 91, 15), labels=[f'{90-a}°' for a in range(0, 91, 15)])

    # CURVA ALTITUD
    ax_cartesian = fig.add_subplot(1, 2, 2)

    #HORA LOCAL
    target_tz = pytz.timezone(tz_name)
    horas_locales = [t.astimezone(target_tz).hour + t.astimezone(target_tz).minute / 60.0 for t in times_utc]

    ax_cartesian.plot(horas_locales, altitudes, color='orange', linewidth=2)
    ax_cartesian.set_title("Altura Solar Durante el Día", pad=20)
    ax_cartesian.set_xlabel(f"Hora Local ({tz_name})")
    ax_cartesian.set_ylabel("Altura Solar (grados)")
    ax_cartesian.set_ylim(0, 90)
    ax_cartesian.set_xlim(min(horas_locales) - 1, max(horas_locales) + 1)
    ax_cartesian.grid(True)
    ax_cartesian.set_xticks(range(0, 25, 2))

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.show()


def iniciar_analisis_solar():
    """Función principal que pide los datos al usuario y ejecuta el análisis."""
    print("\n=============================================")
    print("      BLOQUE 2: ANÁLISIS SOLAR     ")
    print("=============================================")
    try:
        #COORDENADAS
        coords_input = input("➤ Pega las coordenadas a analizar (formato: longitud,latitud): ")
        lon_str, lat_str = coords_input.split(',')
        lat, lon = float(lat_str), float(lon_str)

        #ZONA HORARIA
        tf = TimezoneFinder()
        tz_name = tf.timezone_at(lng=lon, lat=lat) or "UTC"
        print(f"    -> Coordenadas: (Lon={lon:.4f}, Lat={lat:.4f}) | Zona Horaria: {tz_name}")

        #FECHAS
        print("\n--- ☀️ Menú de Fechas ---")
        print("1. Solsticio de Verano\n2. Solsticio de Invierno\n3. Equinoccio\n4. Hoy\n5. Fecha específica (aaaa/mm/dd)")
        opcion = input("➤ Elige una opción: ")

        hoy, year = datetime.now(), datetime.now().year
        fechas = {'1':(year,6,21,"Solsticio de Verano"), '2':(year,12,21,"Solsticio de Invierno"), '3':(year,3,20,"Equinoccio"), '4':(hoy.year,hoy.month,hoy.day,"Hoy")}

        y, m, d, nombre = None, None, None, None
        if opcion in fechas:
            y, m, d, nombre = fechas[opcion]
        elif opcion == '5':
            fecha_obj = datetime.strptime(input("➤ Ingresa la fecha (aaaa/mm/dd): "), '%Y/%m/%d')
            y, m, d, nombre = fecha_obj.year, fecha_obj.month, fecha_obj.day, "Fecha Personalizada"
        else:
            print("Opción no válida."); return

        # GRÁFICO
        posiciones = calcular_posiciones_solares(lat, lon, y, m, d)
        if posiciones:
            graficar_analisis_solar(posiciones, f"{nombre} ({d}/{m}/{y})", lat, tz_name)

    except Exception as e:
        print(f"\n❌ ERROR: Entrada inválida o error en el procesamiento. Asegúrate de usar el formato correcto.\nDetalle: {e}")

#INICIO
iniciar_analisis_solar()

In [ ]:
!pip install skyfield matplotlib numpy pytz timezonefinder pvlib pandas

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
from timezonefinder import TimezoneFinder
import pytz
import pvlib
import pandas as pd
from skyfield.api import Topos, load

#BASE DE DATOS PANELES
PANELES_SOLARES = {
    "1": {"tipo": "Monocristalino", "gamma_pdc": -0.0035},
    "2": {"tipo": "Policristalino", "gamma_pdc": -0.0040},
    "3": {"tipo": "Película Delgada (Thin-Film)", "gamma_pdc": -0.0020}
}

def calcular_y_mostrar_energia():
    """Función principal del Bloque 3: pide datos, calcula y muestra los resultados energéticos."""
    print("\n=============================================")
    print("      ANÁLISIS ENERGÉTICO     ")
    print("=============================================")
    try:
        #RECOLECCIÓN DE DATOS
        coords_input = input("➤ Pega las coordenadas (formato: longitud,latitud): ")
        lon_str, lat_str = coords_input.split(',')
        lat, lon = float(lat_str), float(lon_str)

        elev_input = input("➤ Ingresa la elevación del punto en metros: ")
        elevacion = float(elev_input)

        fecha_input = input("➤ Ingresa la fecha para el análisis (formato aaaa/mm/dd): ").strip().replace('-', '/')
        fecha_obj = datetime.strptime(fecha_input, '%Y/%m/%d')
        y, m, d = fecha_obj.year, fecha_obj.month, fecha_obj.day

        print("\n--- Tipos de Paneles Solares ---")
        for key, value in PANELES_SOLARES.items():
            print(f"{key}. {value['tipo']}")
        panel_choice = input("➤ Elige un tipo de panel: ")

        if panel_choice not in PANELES_SOLARES:
            print("❌ ERROR: Selección de panel no válida."); return

        panel = PANELES_SOLARES[panel_choice]
        potencia_panel = float(input(f"➤ Ingresa la potencia del panel '{panel['tipo']}' (en Vatios, ej: 450): "))

        #CÁLCULOS SOLARES Y DE IRRADIANCIA
        tf = TimezoneFinder()
        tz_name = tf.timezone_at(lng=lon, lat=lat) or "UTC"

        times = pd.date_range(start=f"{y}-{m}-{d}", end=f"{y}-{m}-{d} 23:59:00", freq="5min", tz=tz_name)

        location = pvlib.location.Location(latitude=lat, longitude=lon, tz=tz_name, altitude=elevacion)
        solar_position = location.get_solarposition(times)
        clearsky = location.get_clearsky(solar_position.index, solar_position=solar_position)
        irrad_tracker = pd.DataFrame({'Directa (DNI)': clearsky['dni'], 'Difusa (DHI)': clearsky['dhi']})
        irrad_tracker['Total en Panel'] = irrad_tracker['Directa (DNI)'] + irrad_tracker['Difusa (DHI)']

        # CÁLCULO DE POTENCIA DEL PANEL
        temp_celda = pvlib.temperature.pvsyst_cell(
            poa_global=irrad_tracker['Total en Panel'], temp_air=25, wind_speed=1)

        potencia_generada = pvlib.pvsystem.pvwatts_dc(
            effective_irradiance=irrad_tracker['Total en Panel'], temp_cell=temp_celda,
            pdc0=potencia_panel, gamma_pdc=panel['gamma_pdc'], temp_ref=25.0)

        #VISUALIZACIÓN Y RESUMEN
        print("\nGenerando Gráfico de Irradiancia Solar...")
        fig, ax1 = plt.subplots(figsize=(12, 7))
        clearsky[['dni', 'dhi', 'ghi']].plot(ax=ax1)
        ax1.set_title(f"Irradiancia Solar (Cielo Despejado) - {d}/{m}/{y}")
        ax1.set_ylabel("Irradiancia (W/m²)")
        ax1.set_xlabel(f"Hora Local ({tz_name})")
        ax1.grid(True)
        ax1.legend(['Directa Normal (DNI)', 'Difusa Horizontal (DHI)', 'Global Horizontal (GHI)'])
        plt.show()

        print("\nGenerando Gráfico de Potencia del Panel Solar...")
        fig, ax2 = plt.subplots(figsize=(12, 7))
        potencia_generada.plot(ax=ax2, color='green', label='Potencia Generada (DC)')
        ax2.set_title(f"Potencia Generada por Panel de {potencia_panel}W con Seguidor")
        ax2.set_ylabel("Potencia (Vatios DC)")
        ax2.set_xlabel(f"Hora Local ({tz_name})")
        ax2.grid(True)
        ax2.legend()
        plt.show()

        intervalo_horas = 5 / 60.0
        energia_total_kwh = (potencia_generada.sum() * intervalo_horas) / 1000

        print("\n--- 🔋 Resumen Energético del Día (Cielo Despejado) ---")
        print(f"Panel: {panel['tipo']} de {potencia_panel}W con seguidor de 2 ejes.")
        print(f"Energía total generada: {energia_total_kwh:.2f} kWh")

        # --- NUEVO: Módulo de Cálculo de Ganancias ---
        calcular_ganancias(energia_total_kwh)

    except Exception as e:
        print(f"\n❌ ERROR: Entrada inválida o error en el procesamiento.\nDetalle: {e}")

def calcular_ganancias(energia_kwh):
    """Pide el precio de la energía y calcula las ganancias estimadas."""
    print("\n--- 💰 Módulo de Cálculo de Ganancias ---")
    try:
        moneda = input("➤ Ingresa el símbolo de tu moneda (ej: $, COP, €): ")
        precio_kwh_str = input(f"➤ Ingresa el costo de 1 kWh en {moneda} (ej: 750.5): ")
        precio_kwh = float(precio_kwh_str)

        ganancia_diaria = energia_kwh * precio_kwh
        ganancia_mensual = ganancia_diaria * 30  # Estimación simple
        ganancia_anual = ganancia_diaria * 365 # Estimación simple

        print("\n--- Estimación de Ganancias (Cielo Despejado) ---")
        print(f"Ganancia Diaria:   {moneda} {ganancia_diaria:,.2f}")
        print(f"Ganancia Mensual:  {moneda} {ganancia_mensual:,.2f} (estimado)")
        print(f"Ganancia Anual:    {moneda} {ganancia_anual:,.2f} (estimado)")
        print("\n*Nota: Este es un cálculo idealizado que no considera días nublados o mantenimiento.")

    except ValueError:
        print("\n❌ ERROR: Valor inválido. Por favor, ingresa un número para el precio.")
    except Exception as e:
        print(f"\n❌ Ocurrió un error en el cálculo de ganancias: {e}")

# --- Iniciar el programa del Bloque 3 ---
calcular_y_mostrar_energia()